# GAN

## GAN在优化什么     

`GAN`输入一个正态分布，训练一个生成器$G$，生成器产生出一个分布$P_G$，`GAN`的目的是使生成出的分布$P_G$尽可能接近实际的数据分布$P_{data}$

<img style="float: center;" src="pics/GAN_2_1.PNG" width=600 height=600>

`GAN`的过程表现为下式:

$$G^*=argmin_{G}\ Div(P_G,P_{data})$$

$Div(P_G,P_{data})$代表两个分布之间的散度`(Divergence)`

问题在于，在训练生成器时的`Loss`是可以计算的，但是**散度如何计算？**

## GAN的解决方法

只需要从$P_G$和$P_{data}$中做sampling足够满足,假设$Y$是从$P_G与P_{data}$中sample出来的

<img style="float: center;" src="pics/GAN_2_2.PNG" width=600 height=600>

### 对于Discriminator

目标是分辨出真实数据与生成数据
    
- **GAN的Objective Function**
    根据`GAN`的目标，可以写出网络的优化目标(最大化-目标函数/最小化-损失函数)
    
    $$D^*=arg\ \underset {D}{max} V(D,G)$$

其中损失函数：

$$V(D,G)=E_{y\sim P_{data}}\ (logD(y))+E_{y\sim P_G}\ (1- logD(y))$$

相当于一个**二分类器**，实际上$V(D,G)$的值和**JS散度**有关

### 对于Generator 

目标是在给定判别器的情况下让`Objective function`尽可能大：

$$G^*=arg\ \underset{G}{min}\ \underset{D}{max} V(D,G)$$

### 不同的散度

原始`GAN`中的目标函数的最大值接近`JS散度`，如果需要使用其他散度进行衡量，目标函数设计参考:

`F-GAN:https://arxiv.org/abs/1606.00709`


# Tips for GAN

## JS-divergence 的问题

### 数据的问题


- $P_G$和$P_{data}$重叠部分很少

    1. 图片的自然特性       
    
    图片是高维空间里一个低维的流形(manifold)，即在高维空间中sample一个点，大概率不会是图片，图片在高维空间中只是狭窄的一小部分。
    
    例如在一个二维空间里，$P_G$和$P_{data}$是两条线，那么相交部分几乎可以忽略
    
    <img style="float: center;" src="pics/GAN_2_3.PNG" width=300 height=300>
    
    2. 采样导致的问题
    
    即使$P_G$和$P_{data}$存在非常大的重叠，但如果由于采样的点数量不够多且切不够密集，判别器可以轻松生成一条判别边界区分两个分布。
    
    <img style="float: center;" src="pics/GAN_2_4.PNG" width=300 height=300>
    
### JS-Divergence的衡量

对于没有重叠的两个分布而言，无论两个分布差距多大，`JS散度`都是`log2`。那么意味着，`JS散度`无法对生成器参数进行很好的优化。

<img style="float: center;" src="pics/GAN_2_5.PNG" width=500 height=500>

上图第一个和第二个分布之间的`JS散度`都是`log2`，也就意味着`GAN`没有办法将$P_G$从第一个优化成第二个，二分类器作为判别器而言，Loss很难下降。

## Wasserstein Distance

### Wasserstein距离思想

**Wasserstein距离**的起源是`optimal transport problem`，把概率分布想象成一堆石子，如何移动一堆石子，通过**最小的累积移动距离**把它堆成另外一个**目标分布**，这就是`optimal transport`所关心的问题。

<img style="float: center;" src="pics/GAN_2_6.PNG" width=500 height=500>

但是移动的方法有非常多，因此变为一个优化问题

- **Wasserstein距离**的优点

1. 能够很自然地度量离散分布和连续分布之间的距离；
2. 不仅给出了距离的度量，而且给出如何把一个分布变换为另一分布的方案；
3. 能够连续地把一个分布变换为另一个分布，在此同时，能够保持分布自身的几何形态特征；

如下图，**Wassersteinjuli**相对于**JS距离**而言，第一个分布间的距离比第二个远，那么参数存在优化的可能。

<img style="float: center;" src="pics/GAN_2_7.PNG" width=500 height=500>

### Wasserstein距离公式    

$$V(D,G)=E_{y\sim P_{data}}\ (logD(y))+E_{y\sim P_G}\ (1- logD(y))$$

$$D^*=arg\ \underset {D \in 1-Lipschitz}{max} \ \{E_{y\sim P_{data}}\ (logD(y))-E_{y\sim P_G}\ (logD(y))\}$$

$D \in 1-Lipschitz$可以理解为令判别器函数足够`smooth`